#Case Study 1 : Collecting Data from Twitter

** Due Date: February 10, before the class**

*------------

**TEAM Members:**
    
    Haley Huang
    Helen Hong
    Tom Meagher
    Tyler Reese

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

#Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [1]:
import io
import json
import twitter

#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = '92TpJf8O0c9AWN3ZJjcN8cYxs'
    CONSUMER_SECRET ='dyeCqzI2w7apETbTUvPai1oCDL5oponvZhHSmYm5XZTQbeiygq'
    OAUTH_TOKEN = '106590533-SEB5EGGoyJ8EsjOKN05YuOQYu2rg5muZgMDoNrqN'
    OAUTH_TOKEN_SECRET = 'BficAky6uGyGfRzDGJqZYVKo0HS6G6Ex3ijYW3zy3kjNJ'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------

def save_json(filename, data):
    """
    Save json data to a filename
    """
    with io.open('{0}.json'.format(filename), 
                 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(data, ensure_ascii=False)))

def load_json(filename):
    """
    Load json data from a filename
    """
    with open('{0}.json'.format(filename)) as f:    
        return json.load(f)

twitter_api = oauth_login()
if twitter_api:
    print 'Bingo! API set up!'
else:
    print 'Hmmmm, something is wrong here', twitter_api

topic_of_interest = "jack"
tweets = twitter_api.statuses.user_timeline(screen_name="%s" % topic_of_interest)

# Count number of saved tweets
total_tweet_count = 0
for tweet in tweets: total_tweet_count+=1
print "%s tweets captured from topic of interest, %s." % (total_tweet_count, topic_of_interest)

# Save tweets to file
filename = "tweets"
print 'Saving tweets into {0}.json'.format(filename)
save_json(filename, tweets)

# Load saved tweets into results
results = load_json(filename)
print "Loading tweets into results variable"

Bingo! API set up!
19 tweets captured from topic of interest, jack.
Saving tweets into tweets.json
Loading tweets into results variable


In [5]:
import io
import json
import twitter

def load_json(filename):
    """
    Load json data from a filename
    """
    with open('{0}.json'.format(filename)) as f:    
        return json.load(f)

filename = "tweets"
results = load_json(filename)
print "Loading tweets into results variable"

Loading tweets into results variable


In [24]:
# Compute Additional Statistics about the tweets collected

# Determine the average number of words in the text of each tweet
def average_words(tweet_texts):
    total_words =  sum([len(s.split()) for s in tweet_texts])
    return 1.0*total_words/len(tweet_texts)

tweet_texts = [ tweet['text'] 
                 for tweet in results ]

print 'Average number of words-', average_words(tweet_texts)

# Calculate the lexical diversity of all words contained in the tweets
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens)

words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split() ]

print 'Lexical Diversity-', lexical_diversity(words)


Average number of words- 13.8947368421
Lexical Diversity- 0.795454545455


###Report some statistics about the tweets you collected 

*The topic of interest: jack

*The total number of tweets collected:  19

*-----------------------

#Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [2]:
from collections import Counter
from prettytable import PrettyTable

tweet_texts = [ tweet['text'] 
                 for tweet in results ]
words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split() ]

# frequency of words
count = Counter(words).most_common()

# table of the top 30 words with their counts
pretty_table = PrettyTable(field_names=['Word', 'Count']) 
[ pretty_table.add_row(w) for w in count[:30] ]
pretty_table.align['Word'], pretty_table.align['Count'] = 'l', 'r' # Set column alignment
print pretty_table

+-------------+-------+
| Word        | Count |
+-------------+-------+
| RT          |    11 |
| to          |     8 |
| way         |     4 |
| for         |     3 |
| on          |     3 |
| your        |     3 |
| is          |     3 |
| you         |     3 |
| the         |     3 |
| just        |     2 |
| follow      |     2 |
| only        |     2 |
| excited     |     2 |
| new         |     2 |
| #IowaCaucus |     2 |
| our         |     2 |
| out         |     2 |
| @Square     |     2 |
| Iowa        |     2 |
| about       |     2 |
| of          |     2 |
| The         |     2 |
| more        |     2 |
| with        |     2 |
| this        |     2 |
| see         |     2 |
| can         |     2 |
| in          |     2 |
| if          |     2 |
| that        |     2 |
+-------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [22]:
from collections import Counter
from prettytable import PrettyTable

# Create a list of all tweets with at least one retweet, and index the originator of that tweet and the text.
retweets = [
            (tweet['retweet_count'],
            tweet['retweeted_status']['user']['screen_name'],
            tweet['text'])
            
            #Ensure that a retweet exists
            for tweet in results                      
                if tweet.has_key('retweeted_status')
            ]

pt = PrettyTable(field_names = ['Count','Screen Name','Text'])

# Sort tweets by descending number of retweets and display the top 10 results in a table.
[pt.add_row(row) for row in sorted(retweets, reverse = True)[:10]]
pt.max_width['Text'] = 50
pt.align = 'l'
print pt

+-------+--------------+----------------------------------------------------+
| Count | Screen Name  | Text                                               |
+-------+--------------+----------------------------------------------------+
| 2863  | vine         | RT @vine: ☁️☁️☁️  https://t.co/ZjshUKxSw3          |
| 893   | blackbirds   | RT @blackbirds: We're excited to celebrate Black   |
|       |              | History Month with a new emoji. Join the           |
|       |              | conversation all month long with #BlackHistoryMon… |
| 157   | gomery       | RT @gomery: Currently in parts of Iowa.            |
|       |              | #IowaCaucus  https://t.co/QRNo0LznxE               |
| 80    | NickPacilio  | RT @NickPacilio: Reminder - Iowans were Tweeting   |
|       |              | about @tedcruz more than any other GOP candidate   |
|       |              | this past week. https://t.co/XIGOGjgoAr            |
| 55    | adambain     | RT @adambain: We’re expanding our logge

Another measure of tweet "popularity" could be the number of times it is favorited.  The following calculates the top-10 tweets with the most "favorites"

In [28]:
# Determine the number of "favorites" for each tweet collected.

favorites = [
            (tweet['favorite_count'],
             tweet['text'])
            for tweet in results
            ]
            
pt = PrettyTable(field_names = ['Count','Text'])

# Sort tweets by descending number of favorites and display the top 10 results in a table.
[pt.add_row(row) for row in sorted(favorites, reverse = True)[:10]]
pt.max_width['Text'] = 50
pt.align = 'l'
print pt

+-------+----------------------------------------------------+
| Count | Text                                               |
+-------+----------------------------------------------------+
| 375   | Fast way to see what's trending now📈 on iPhone    |
|       | 6s: 3D Touch (press in) the Twitter icon and tap   |
|       | search! https://t.co/6p6UIDnZU3                    |
| 353   | Trends have more context now so you can see if     |
|       | someone you follow is tweeting about it            |
|       | https://t.co/A28Ubklf9q                            |
| 180   | 💯💯💯 https://t.co/vU0ne0wEHK                     |
| 143   | "Twitter nails it." #IowaCaucus                    |
|       | https://t.co/yMaJPOlJNZ                            |
| 133   | Watch Iowa caucus voting live:                     |
|       | https://t.co/tU98z67jI5                            |
| 122   | This is a fascinating way to experience an         |
|       | election year. Right on the ground live where it'

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [21]:
#----------------------------------------------

# Extract the scree names which appear among the collection of tweets
screen_names = [user_mention['screen_name']
               for tweet in results
                   for user_mention in tweet['entities']['user_mentions']]

# Extract the hashtags which appear among the collection of tweets
hashtags = [ hashtag['text']
           for tweet in results
               for hashtag in tweet['entities']['hashtags']]

# Simultaneously determine the frequency of screen names/hashtags, and display the top 10 most common in a table.
for label, data in (('Screen Name',screen_names),
                   ('Hashtag',hashtags)):
    pt = PrettyTable(field_names =[label,'Count'])
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:10]]
    pt.align[label],pt.align['Count'] = 'l','r'
    print pt


+--------------+-------+
| Screen Name  | Count |
+--------------+-------+
| Square       |     2 |
| ThePayam     |     2 |
| eyeseewaters |     1 |
| patrickc     |     1 |
| MAKERSwomen  |     1 |
| NickPacilio  |     1 |
| blackbirds   |     1 |
| tedcruz      |     1 |
| vine         |     1 |
| periscopetv  |     1 |
+--------------+-------+
+-------------------+-------+
| Hashtag           | Count |
+-------------------+-------+
| IowaCaucus        |     2 |
| KeepPounding      |     1 |
| Broncos           |     1 |
| SuperBowl         |     1 |
| GroundhogDay      |     1 |
| MAKERS2016        |     1 |
| BlackHistoryMonth |     1 |
| TheTimeIsNow      |     1 |
| SB50              |     1 |
| BHM               |     1 |
+-------------------+-------+


*------------------------

#Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [29]:
#----------------------------------------------
# The following is the "general-purpose API wrapper" presented in "Mining the social Web" for making robust twitter requests.
import sys
import time
from urllib2 import URLError
from httplib import BadStatusLine
import json

# This function can be used to accompany any twitter API function.  It force-breaks after receiving more than max_errors
# error messages from the Twitter API.

def make_twitter_request(twitter_api_func, max_errors = 10, *args, **kw):
    def handle_twitter_http_error(e, wait_period = 2, sleep_when_rate_limited = True):
        
        if wait_period > 3600:
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
        
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        
        elif e.e.code == 429:
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying again in 15 Minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e
                
        elif e.e.code in (500,502,503,504):
            print >> sys.stderr, 'Encountered %i Error.  Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleel(wait.period)
            wait.period *= 1.5
            return wait_period
        
        else:
            raise e
            
    wait_period = 2
    error_count = 0
    
    while True:
        try:
            return twitter_api_func(*args,**kw)
        except twitter.api.TwitterHTTPError, e:
            error_count = 0
            wait_period = handle_twitter_http_error (e, wait_period)
            if wait_period is None:
                return
            
        except URLError, e:
            error_count += 1
            print >> sys.stderr, "URLError encountered.  Continuing"
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        
        except BadStatusLine, e:
            error_count += 1
            print >> sys.stderr, "BadStatusLineEncountered.  Continuing"
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

In [ ]:
# This function uses the above Robust Request wrapper to retreive all friends and followers of a given user.  This code
# can be found in Chapter 9, the `Twitter Cookbook' in "Mining the social web"

def get_friends_followers_ids(twitter_api, screen_name = None, user_id = None, friends_limit = maxint, followers_limit = maxint):
    assert(screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids,
                             count = 5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids,
                               count = 5000)
    
    friends_ids, followers_ids = [],[]
    
    for twitter_api_func, limit, ids, label in[
        [get_friends_ids, friends_limit, friends_ids, "friends"],
        [get_followers_ids, followers_limit, followers_ids, "followers"]
           ]:
        if limit ==0: continue
        
        cursor = -1
        while cursor != 0:
            
            if screen_name:
                response = twitter_api_func(screen_name = screen_name, cursor = cursor)
            else:
                response = twitter_api_func(user_id = user_id, cursor = cursor)
                
            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
                
            print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids),label,(user_id or screen_name))
            
            if len(ids) >= limit or response is None:
                break
                
    return friends_ids[:friends_limit], followers_ids[:followers_limit]

#--------------------------------
# Retrieve the friends and followers of a user and save to a json file.

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, screen_name = 'User')
    
save_json('User_Followers',followers_ids)
save_json('User_Friends', friends_ids)

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [ ]:
# This function views the friend and follower lists computed above as mathematical sets.  It then computes the set-intersection 
# to determine mutual friends.  It also uses set-differences to determine unbalanced friendships.  It returns to the user:
    # Number of friends
    # Number of followers
    # Number of mutual friends
    # Number of friens which are not followers
    # Number of followers which are not friends
    
def setwise_friends_followers_analysis(screen_name, friends_ids, followers_ids):
    friends_ids, followers_ids = set(friends_ids),set(followers_ids)
    
    print '{0} is following {1}'.format(screen_name, len(friends_ids))
    
    print '{0} is being followed by {1}'.format(screen_name,len(followers_ids))
    print '{0} has {1} mutual friends'.format(screen_name, len(friends_ids.intersection(followers_ids)))
    print'{0} of {1} are not following {2} back'.format(len(friends_ids.difference(followers_ids)),len(friends_ids),screen_name)
    print '{0} of {1} are not being followed back by {2}'.format(len(followers_ids.difference(friends_ids)),len(followers_ids),screen_name)


setwise_friends_followers_analysis('User',friends_ids,followers_ids)





*------------------------

#Problem 4: Explore the data 

Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API

In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through myWPI, in the Assignment "Case Study 1".
        
** Note: Each team just need to submit one submission in myWPI **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Explore the data
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

